# Лабораторная работа 1. Введение в машинное обучение. Обучение с учителем. Задача регрессии

<b>Традиционное предупреждение для всех лабораторных работ:</b> перед обучением моделей необходимо выполнить предварительную обработку данных, которая <b>обязательно</b> включает в себя:
- заполнение пропущенных значений (рекомедуется логика заполнения пропусков на основе типа данных, которая использовалась в РГР по Практикуму);
- преобразование категориальных признаков в числовые (используйте one-hot кодирование или map; используйте знания с Практикума).

Предобработка может включать в себя другие действия, но выполнение описанных выше действий обязательно.

Сделайте это один раз и сохраните в отдельный csv файл, а потом его используйте.

<b>Выполните следующие задания:</b>
- загрузите датасет для регрессии, выделите целевой признак и предикторы, разбейте данные на обучающую и тестовую выборку;
- решите задачу регрессии на ваших данных с использованием моделей sklearn (линейная регрессия + L1, L2), для моделей с регуляризациями подберите гиперпараметр;
- решите задачу регрессии на ваших данных с использованием моделей sklearn (полиномиальная регрессия + L1, L2), для моделей с регуляризациями подберите гиперпараметр;
- вычислите значения метрик $R^2$, MAE, MSE, RMSE, MAPE для всех обученных моделей; выберите лучшую модель;
- самостоятельно реализуйте (желательно в виде класса) модель линейной регрессии с регуляризацией (можете выбрать L1 или L2);
- самостоятельно реализуйте вычисление всех используемых метрик (в виде функций, принимающих два аргумента);
- обучите вашу модель линейной регрессии на ваших данных; оцените качество с помощью реализованных вами метрик.

In [139]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from math import sqrt
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt

data = pd.read_csv('result.csv', sep = ',')
data = data.drop(columns = ['Unnamed: 0'])
data

,price,area,latitude,longitude,Bedrooms,Bathrooms,Balcony,parking,Lift,Ready to Move,Under Construction,New Property,Resale,Furnished,Semi-Furnished,Unfurnished,Flat,Individual House
0,22400000.0,629.0,19.032800,72.896357,2.0,2.0,0.0,0.0,0.0,0,1,1,0,0,0,1,1,0
1,35000000.0,974.0,19.032800,72.896357,3.0,2.0,0.0,0.0,0.0,0,1,1,0,0,0,1,1,0
2,31700000.0,968.0,19.085600,72.909277,3.0,3.0,0.0,0.0,0.0,0,1,1,0,0,0,1,1,0
3,18700000.0,629.0,19.155756,72.846862,2.0,2.0,2.0,2.0,2.0,1,0,1,0,0,0,1,1,0
4,13500000.0,1090.0,19.177555,72.849887,2.0,2.0,0.0,0.0,0.0,1,0,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6250,19500000.0,810.0,19.138320,72.810020,2.0,2.0,0.0,0.0,0.0,1,0,0,1,0,1,0,1,0
6251,22000000.0,1400.0,19.221920,72.854250,3.0,3.0,1.0,1.0,1.0,1,0,0,1,0,0,1,1,0
6252,20000000.0,750.0,19.144320,72.824111,2.0,2.0,0.0,0.0,0.0,1,0,0,1,0,0,1,1,0
6253,11000000.0,700.0,19.047201,72.872225,2.0,2.0,0.0,0.0,0.0,1,0,0,1,1,0,0,1,0


In [140]:
y = data['price']
X = data.drop(['price'], axis = 1)

In [141]:
y

0       22400000.0
1       35000000.0
2       31700000.0
3       18700000.0
4       13500000.0
           ...    
6250    19500000.0
6251    22000000.0
6252    20000000.0
6253    11000000.0
6254    15000000.0
Name: price, Length: 6255, dtype: float64

In [142]:
X

,area,latitude,longitude,Bedrooms,Bathrooms,Balcony,parking,Lift,Ready to Move,Under Construction,New Property,Resale,Furnished,Semi-Furnished,Unfurnished,Flat,Individual House
0,629.0,19.032800,72.896357,2.0,2.0,0.0,0.0,0.0,0,1,1,0,0,0,1,1,0
1,974.0,19.032800,72.896357,3.0,2.0,0.0,0.0,0.0,0,1,1,0,0,0,1,1,0
2,968.0,19.085600,72.909277,3.0,3.0,0.0,0.0,0.0,0,1,1,0,0,0,1,1,0
3,629.0,19.155756,72.846862,2.0,2.0,2.0,2.0,2.0,1,0,1,0,0,0,1,1,0
4,1090.0,19.177555,72.849887,2.0,2.0,0.0,0.0,0.0,1,0,1,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6250,810.0,19.138320,72.810020,2.0,2.0,0.0,0.0,0.0,1,0,0,1,0,1,0,1,0
6251,1400.0,19.221920,72.854250,3.0,3.0,1.0,1.0,1.0,1,0,0,1,0,0,1,1,0
6252,750.0,19.144320,72.824111,2.0,2.0,0.0,0.0,0.0,1,0,0,1,0,0,1,1,0
6253,700.0,19.047201,72.872225,2.0,2.0,0.0,0.0,0.0,1,0,0,1,1,0,0,1,0


In [143]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

In [144]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((5004, 17), (5004,), (1251, 17), (1251,))

In [145]:
lr = LinearRegression().fit(X_train, y_train)
y_pred = lr.predict(X_test)
y_pred

array([27177670.20618057, 38845378.00874138, 58187894.17677498, ...,
       18345903.54318237, 23899884.27685452, 11439649.96230125])

In [146]:
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {lr.score(X_test, y_test)}')

MAE: 10231528.734740173
MSE: 326239809616389.3
RMSE: 18062109.777553376
MAPE: 0.7513574051424674
R^2: 0.640183993102154


In [147]:
len(lr.coef_)
lr.coef_

array([ 2.13771508e+04, -7.90092497e+07, -7.28394540e+07,  6.84026190e+05,
        7.30302657e+06, -1.33150346e+05, -1.33150346e+05, -1.33150346e+05,
       -2.09529614e+06,  2.09529614e+06,  7.06370378e+04, -7.06370378e+04,
        1.54892456e+06, -6.43694409e+05, -9.05230150e+05,  5.12404739e+03,
       -5.12404739e+03])

In [161]:
#L1
ridge = Ridge(alpha = 0.5).fit(X_train, y_train)
y_pred = ridge.predict(X_test)
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {ridge.score(X_test, y_test)}')
ridge.coef_

MAE: 10213691.98193858
MSE: 326572689206124.25
RMSE: 18071322.287152212
MAPE: 0.7489560803228466
R^2: 0.6398168539571887


array([ 2.13996964e+04, -7.82397249e+07, -7.07962894e+07,  6.71239348e+05,
        7.33044081e+06, -1.37101776e+05, -1.37101776e+05, -1.37101776e+05,
       -1.94383704e+06,  1.94383704e+06,  7.32798447e+04, -7.32798447e+04,
        1.56636736e+06, -6.56597046e+05, -9.09770312e+05,  1.86416172e+04,
       -1.86416172e+04])

In [163]:
#L2
lasso = Lasso(alpha = 0.5).fit(X_train, y_train)
y_pred = ridge.predict(X_test)
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {sqrt(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {lasso.score(X_test, y_test)}')
lasso.coef_

MAE: 10213691.98193858
MSE: 326572689206124.25
RMSE: 18071322.287152212
MAPE: 0.7489560803228466
R^2: 0.6401839672614689


array([ 2.13771528e+04, -7.90092062e+07, -7.28393093e+07,  6.84025157e+05,
        7.30302715e+06, -3.99451753e+05, -0.00000000e+00, -0.00000000e+00,
       -4.18975859e+06,  1.22309415e-09,  1.41273382e+05, -0.00000000e+00,
        2.42377011e+06,  2.31149243e+05, -3.03828178e+04,  1.02414445e+04,
       -2.58335126e-09])

In [164]:
parameters = {'alpha' : np.arange(0, 1, 0.1)}
ridge_optimal = GridSearchCV(Ridge(), parameters).fit(X_train, y_train)
ridge_optimal.best_params_

{'alpha': 0.7000000000000001}

In [151]:
ridge_optimal = RandomizedSearchCV(Ridge(), parameters).fit(X_train, y_train)
ridge_optimal.best_params_

{'alpha': 0.7000000000000001}

In [152]:
#Полиноми...
p = PolynomialFeatures(2)
X_p= p.fit_transform(X)
X_p

array([[  1.       , 629.       ,  19.0327996, ...,   1.       ,
          0.       ,   0.       ],
       [  1.       , 974.       ,  19.0327996, ...,   1.       ,
          0.       ,   0.       ],
       [  1.       , 968.       ,  19.0856   , ...,   1.       ,
          0.       ,   0.       ],
       ...,
       [  1.       , 750.       ,  19.1443196, ...,   1.       ,
          0.       ,   0.       ],
       [  1.       , 700.       ,  19.0472012, ...,   1.       ,
          0.       ,   0.       ],
       [  1.       , 900.       ,  19.1688179, ...,   1.       ,
          0.       ,   0.       ]])

In [153]:
lr2 = LinearRegression().fit(X_p, y)
lr2.coef_

array([ 7.23171244e+06,  2.77699801e+06, -1.02792509e+10, -1.02053424e+10,
       -6.63383042e+08,  9.53743987e+08, -3.93747220e+05, -3.93744201e+05,
       -3.93743193e+05,  8.28038822e+07, -8.28038772e+07, -7.82674491e+07,
        7.82674642e+07,  1.20815904e+08,  9.87352153e+06, -1.30689371e+08,
        1.04492731e+08, -1.04492706e+08, -1.32921944e+00, -6.69745589e+04,
       -1.03085430e+05,  2.17699490e+03,  1.24234971e+03,  2.45725731e+01,
        1.98184580e+01,  1.92554514e+01,  2.33101733e+06,  4.44646109e+05,
        1.38789287e+06,  1.38777047e+06,  9.28113055e+05,  9.22253281e+05,
        9.25299084e+05,  1.38442175e+06,  1.39124083e+06,  2.35095131e+07,
        4.37613337e+08, -2.01963299e+05, -3.02587934e+07,  7.97633705e+05,
        7.97633696e+05,  7.97633698e+05, -8.72750105e+09, -1.55174989e+09,
       -5.13770384e+09, -5.14154709e+09, -3.43606514e+09, -3.42323622e+09,
       -3.41994958e+09, -5.14491448e+09, -5.13433646e+09,  1.34654452e+08,
        3.03802993e+07, -

In [154]:
X_d = X
y_d = lr2.coef_[0][2] + lr2.coef_[0][1]*X_d + lr2.coef_[0][0]*X_d**2
y_d

IndexError: invalid index to scalar variable.

In [155]:
X_p.shape

(6255, 171)

In [156]:
X.shape

(6255, 17)

In [ ]:
class LinRegression:
    def __init__(self, X_train, y_train):
        self.X_train = X_tain
        self.y_train = y_train
        
        
        
        